In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('display.max_columns', None)


In [2]:
#Conexao a db
database = 'database.sqlite'
con = sqlite3.connect(database)

In [3]:
#Get main tables from db
countries = pd.read_sql_query("SELECT * from Country", con)
matches = pd.read_sql_query("SELECT * from Match", con)
leagues = pd.read_sql_query("SELECT * from League", con)
teams = pd.read_sql_query("SELECT * from Team", con)
teams_att = pd.read_sql_query("SELECT * FROM Team_Attributes", con)

In [4]:
#Replace league_id for its name to make it easier to analyse the data.
league_id_map = leagues.set_index('id').to_dict()['name']
matches['league_id'] = matches['league_id'].map(league_id_map)

#Replace country_id for its name.
country_id_map = countries.set_index('id').to_dict()['name']
matches['country_id'] = matches['country_id'].map(country_id_map)

#Replace team_id for its name.
team_id_map = teams.set_index('team_api_id').to_dict()['team_long_name']
matches['home_team_api_id'] = matches['home_team_api_id'].map(team_id_map)
matches['away_team_api_id'] = matches['away_team_api_id'].map(team_id_map)

In [5]:
# Goals avg per league
avg_goals_league = matches[['league_id','home_team_goal','away_team_goal']].groupby('league_id').mean()


total_goals = avg_goals_league.home_team_goal + avg_goals_league.away_team_goal

avg_goals_league.home_team_goal = avg_goals_league.home_team_goal / total_goals * 100
avg_goals_league.away_team_goal = avg_goals_league.away_team_goal / total_goals * 100

avg_goals_league

# Conclusion: home teams have advantage

,home_team_goal,away_team_goal
league_id,,
Belgium Jupiler League,57.446809,42.553191
England Premier League,57.220874,42.779126
France Ligue 1,57.425609,42.574391
Germany 1. Bundesliga,56.060819,43.939181
Italy Serie A,57.352755,42.647245
Netherlands Eredivisie,57.769822,42.230178
Poland Ekstraklasa,57.517182,42.482818
Portugal Liga ZON Sagres,55.566237,44.433763
Scotland Premier League,54.267277,45.732723


In [6]:
# Home win 

goals_per_match = matches[['home_team_goal','away_team_goal']]
wins = 0
draws = 0

for index, row in goals_per_match.iterrows():
    if row['home_team_goal'] > row['away_team_goal']:
        wins += 1
    if row['home_team_goal'] == row['away_team_goal']:
        draws += 1
        

total_matches = len(goals_per_match)
loss = total_matches - wins - draws

percs = [wins/total_matches*100,draws/total_matches*100, loss/total_matches*100]
percs

#Conclusion : this confirms that "home team wins about 46% of the time"

[45.87166557604219, 25.389737865198814, 28.738596558759]

In [51]:
#Cleaning matches
matches_df = pd.read_sql("""SELECT * from MATCH""", con)
teams_df = pd.read_sql("""SELECT * from TEAM""", con)
player_attributes_df = pd.read_sql("""SELECT * from PLAYER_ATTRIBUTES""", con)

matches_df['date'] = pd.to_datetime(matches_df['date'], format='%Y-%m-%d')

home_players = ["home_player_" + str(x) for x in range(1, 12)]
away_players = ["away_player_" + str(x) for x in range(1, 12)]

matches_kept_columns = ["id", "date", "home_team_api_id", "away_team_api_id", "home_team_goal", "away_team_goal"]
matches_kept_columns = matches_kept_columns + home_players
matches_kept_columns = matches_kept_columns + away_players

matches_df = matches_df[matches_kept_columns]

matches_df['game_status'] = 0
matches_df['game_status'] = np.where(matches_df['home_team_goal'] > matches_df['away_team_goal'] , '1', matches_df['game_status'])
matches_df['game_status'] = np.where(matches_df['home_team_goal'] < matches_df['away_team_goal'], '-1', matches_df['game_status'])

for player in home_players:
    matches_df = pd.merge(matches_df, player_attributes_df[["id", "overall_rating"]], left_on=[player], right_on=["id"], suffixes=["", "_" + player])
for player in away_players:
    matches_df = pd.merge(matches_df, player_attributes_df[["id", "overall_rating"]], left_on=[player], right_on=["id"], suffixes=["", "_" + player])
    
matches_df = matches_df.rename(columns={"overall_rating": "overall_rating_home_player_1"})

matches_df = matches_df[ matches_df[['overall_rating_' + p for p in home_players]].isnull().sum(axis = 1) <= 0]
matches_df = matches_df[ matches_df[['overall_rating_' + p for p in away_players]].isnull().sum(axis = 1) <= 0]

matches_df['overall_rating_home'] = matches_df[['overall_rating_' + p for p in home_players]].sum(axis=1)
matches_df['overall_rating_away'] = matches_df[['overall_rating_' + p for p in away_players]].sum(axis=1)
matches_df['overall_rating_difference'] = matches_df['overall_rating_home'] - matches_df['overall_rating_away']

matches_df['mean_overall_rating_home'] = matches_df[['overall_rating_' + p for p in home_players]].mean(axis=1)
matches_df['mean_overall_rating_away'] = matches_df[['overall_rating_' + p for p in away_players]].mean(axis=1)

for c in matches_df.columns:
    if '_player_' in c:
        matches_df = matches_df.drop(c, axis=1)
        
#Replace team_id for its name.
team_id_map = teams_df.set_index('team_api_id').to_dict()['team_long_name']
matches_df['home_team_api_id'] = matches_df['home_team_api_id'].map(team_id_map)
matches_df['away_team_api_id'] = matches_df['away_team_api_id'].map(team_id_map)
        

In [54]:
pd.read_sql("""SELECT IWD, IWA, LBH, LBD, LBA, PSH, PSD, PSA,
WHH,
WHD,
WHA,
SJH,
SJD,
SJA,
VCH,
VCD,
VCA,
GBH,
GBD,
GBA,
BSH,
BSD,
BSA from MATCH WHERE NOT IWD='NaN'""", con)

,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,3.2,3.50,1.80,3.30,3.75,NaN,NaN,NaN,1.70,3.30,4.33,1.90,3.3,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,3.2,3.50,1.90,3.20,3.50,NaN,NaN,NaN,1.83,3.30,3.60,1.95,3.3,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3.1,2.30,2.50,3.20,2.50,NaN,NaN,NaN,2.50,3.25,2.40,2.63,3.3,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,3.9,6.00,1.44,3.60,6.50,NaN,NaN,NaN,1.44,3.75,6.00,1.44,4.0,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,3.3,1.70,4.00,3.40,1.72,NaN,NaN,NaN,4.20,3.40,1.70,4.50,3.5,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22515,3.7,4.90,1.57,3.75,6.50,1.58,4.05,7.06,1.62,3.30,7.00,NaN,NaN,NaN,1.57,4.00,7.00,NaN,NaN,NaN,NaN,NaN,NaN
22516,3.2,3.20,2.25,3.20,3.25,2.36,3.36,3.36,2.38,3.10,3.10,NaN,NaN,NaN,2.30,3.40,3.40,NaN,NaN,NaN,NaN,NaN,NaN
22517,3.7,5.40,1.50,4.00,6.50,1.55,4.31,6.95,1.57,3.50,7.00,NaN,NaN,NaN,1.55,4.20,7.00,NaN,NaN,NaN,NaN,NaN,NaN
22518,3.3,2.75,2.30,3.20,3.10,2.34,3.40,3.35,2.40,3.10,3.10,NaN,NaN,NaN,2.30,3.40,3.30,NaN,NaN,NaN,NaN,NaN,NaN


,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,Belgium,Belgium Jupiler League,2008/2009,1,2008-08-17 00:00:00,492473,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.73,3.40,5.00,1.75,3.35,4.20,1.85,3.2,3.5,1.80,3.3,3.75,NaN,NaN,NaN,1.70,3.30,4.33,1.90,3.3,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,Belgium,Belgium Jupiler League,2008/2009,1,2008-08-16 00:00:00,492474,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.95,3.20,3.60,1.80,3.30,3.95,1.90,3.2,3.5,1.90,3.2,3.50,NaN,NaN,NaN,1.83,3.30,3.60,1.95,3.3,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,Belgium,Belgium Jupiler League,2008/2009,1,2008-08-16 00:00:00,492475,NaN,NaN,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,2.38,3.30,2.75,2.40,3.30,2.55,2.60,3.1,2.3,2.50,3.2,2.50,NaN,NaN,NaN,2.50,3.25,2.40,2.63,3.3,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,Belgium,Belgium Jupiler League,2008/2009,1,2008-08-17 00:00:00,492476,NaN,NaN,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.44,3.75,7.50,1.40,4.00,6.80,1.40,3.9,6.0,1.44,3.6,6.50,NaN,NaN,NaN,1.44,3.75,6.00,1.44,4.0,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,Belgium,Belgium Jupiler League,2008/2009,1,2008-08-16 00:00:00,492477,NaN,NaN,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,5.00,3.50,1.65,5.00,3.50,1.60,4.00,3.3,1.7,4.00,3.4,1.72,NaN,NaN,NaN,4.20,3.40,1.70,4.50,3.5,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
...,...,...,...,...,...,...,...,...,...,...,...,.